# Selenium ile Veri Otomasyonu

**Hedefler**
1. BART API kullanarak istasyonların enlemlerinin ve boylamlarının olduğu json dosyasını çekeceğiz
2. Elimizde bulunan bu verilerle istasyonlar arası mesafeyi hesaplamak için tarimziraat.com isimli siteyi kullanacağız.

![image](station.png)

# Kütüphaneler

In [1]:
import pandas as pd # Veriseti haline getirmek için kullanacağımız kütüphane
import seaborn as sns 
import warnings    # Hata mesajlarını görmemek için bu kütüphaneyi de dahil ediyoruz
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings("ignore")  # Hata mesajlarını görmezden gel komutu

In [3]:
from bs4 import BeautifulSoup   # İnternetten çekeceğimiz verileri html formatında düzenli
                                # çekebilmek için kullanacağımız kütüphane
    
import requests                 # İnternet sitesine istek atabilmek için kullanacağımız kütüphane
import json                     # json dosya formatını düzenlemek için kullanılacak kütüphane

## Enlem ve Boylam Verileri

In [6]:
url="http://api.bart.gov/api/stn.aspx?cmd=stns&key=MW9S-E7SL-26DU-VV8V&json=y"
response=requests.get(url).json()

In [7]:
stations = response['root']['stations']['station']
gtfs_latitude = []
gtfs_longitude = []
abbr = []
for i in stations:
    gtfs_latitude.append(i['gtfs_latitude'])
    gtfs_longitude.append(i['gtfs_longitude'])
    abbr.append(i['abbr'])

dictionary = {'Abbreviation':abbr, 'Latitude':gtfs_latitude, 'Longitude':gtfs_longitude}
df = pd.DataFrame(dictionary)

In [10]:
df.head(15) # Şu anda elimizde 50 adet istasyonun enlem ve boylam verisi var

,Abbreviation,Latitude,Longitude
0,12TH,37.803768,-122.271450
1,16TH,37.765062,-122.419694
2,19TH,37.808350,-122.268602
3,24TH,37.752470,-122.418143
4,ANTC,37.995388,-121.780420
5,ASHB,37.852803,-122.270062
6,BALB,37.721585,-122.447506
7,BAYF,37.696924,-122.126514
8,BERY,37.368473,-121.874681
9,CAST,37.690746,-122.075602


**Bu istasyonlar arası kombinasyonları yani birbirleri arasındaki mesafeyi hesaplayabilmek için bunları hepsinden oluşabilecek kombinasyonları bulmamız gerek**

In [11]:
import itertools

In [12]:
routes = pd.DataFrame()

routes['permutations']=list(itertools.permutations(df.values,2))
routes['permutations']

0       ([12TH, 37.803768, -122.271450], [16TH, 37.765...
1       ([12TH, 37.803768, -122.271450], [19TH, 37.808...
2       ([12TH, 37.803768, -122.271450], [24TH, 37.752...
3       ([12TH, 37.803768, -122.271450], [ANTC, 37.995...
4       ([12TH, 37.803768, -122.271450], [ASHB, 37.852...
                              ...                        
2445    ([WOAK, 37.804872, -122.295140], [SSAN, 37.664...
2446    ([WOAK, 37.804872, -122.295140], [UCTY, 37.590...
2447    ([WOAK, 37.804872, -122.295140], [WCRK, 37.905...
2448    ([WOAK, 37.804872, -122.295140], [WARM, 37.502...
2449    ([WOAK, 37.804872, -122.295140], [WDUB, 37.699...
Name: permutations, Length: 2450, dtype: object

In [14]:
liste=list()
for i in routes['permutations']:
    liste.append(i[0][0]+" -> "+i[1][0])

routes['route'] = liste

In [15]:
routes['route']

0       12TH -> 16TH
1       12TH -> 19TH
2       12TH -> 24TH
3       12TH -> ANTC
4       12TH -> ASHB
            ...     
2445    WOAK -> SSAN
2446    WOAK -> UCTY
2447    WOAK -> WCRK
2448    WOAK -> WARM
2449    WOAK -> WDUB
Name: route, Length: 2450, dtype: object

**Bu işlemler sayesinde herbir istasyonun arasında oluşabilecek kombinasyonları bulmuş olduk, bu da elimizde 2450 adet hesaplanması gereken işlem verdi**

In [16]:
routes

,permutations,route
0,"([12TH, 37.803768, -122.271450], [16TH, 37.765...",12TH -> 16TH
1,"([12TH, 37.803768, -122.271450], [19TH, 37.808...",12TH -> 19TH
2,"([12TH, 37.803768, -122.271450], [24TH, 37.752...",12TH -> 24TH
3,"([12TH, 37.803768, -122.271450], [ANTC, 37.995...",12TH -> ANTC
4,"([12TH, 37.803768, -122.271450], [ASHB, 37.852...",12TH -> ASHB
...,...,...
2445,"([WOAK, 37.804872, -122.295140], [SSAN, 37.664...",WOAK -> SSAN
2446,"([WOAK, 37.804872, -122.295140], [UCTY, 37.590...",WOAK -> UCTY
2447,"([WOAK, 37.804872, -122.295140], [WCRK, 37.905...",WOAK -> WCRK
2448,"([WOAK, 37.804872, -122.295140], [WARM, 37.502...",WOAK -> WARM


# Selenium

In [17]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

path = r"Bu kısma kendi bilgisayarınızdaki chrom driver'ının yolunu eklemeniz gerek"

In [22]:
driver = webdriver.Chrome(r"C:\Users\MONSTERHAN\Documents\GitHub\MachineLearning-WebScrapping-NLP\TR\Selenium-ile-Veri-Otomasyonu\chromedriver.exe")
driver.get("https://www.tarimziraat.com/tarimsal_harita_uygulamalari/haritadan_iki_nokta_arasi_olcumu.php")
time.sleep(2)

distance= list()

for i in routes['permutations']:
    
    enlem1=driver.find_element_by_xpath('//*[@id="enlem1"]').clear()
    enlem2=driver.find_element_by_xpath('//*[@id="enlem2"]').clear()
    boylam1=driver.find_element_by_xpath('//*[@id="boylam1"]').clear()
    boylam2=driver.find_element_by_xpath('//*[@id="boylam2"]').clear()
    
    enlem1=driver.find_element_by_xpath('//*[@id="enlem1"]').send_keys(i[0][1])
    enlem2=driver.find_element_by_xpath('//*[@id="enlem2"]').send_keys(i[1][1])
    boylam1=driver.find_element_by_xpath('//*[@id="boylam1"]').send_keys(i[0][2])
    boylam2=driver.find_element_by_xpath('//*[@id="boylam2"]').send_keys(i[1][2])
    buton1=driver.find_element_by_xpath('//*[@id="buton2"]')
    buton1.click()
    sonuc = driver.find_element_by_xpath('//*[@id="sonuc"]/b')
    distance.append(sonuc.text)
routes['distances'] = distance

**Bu kodları çalıştırdığımızda (selenium direver dosyası bilgisayarınızda olması gerekiyor) bir browser dosyası açılacak ve otomatik bir şekilde enlem ve boylamlar kullanılarak routes isimli veri setindeki yeni bir sütuna karşılıları yazılıyor**

![image](driver.png)

In [27]:
routes.to_csv('pause.csv')  # Burada bir yedekleme yaptım çünkü az önce yaptığımız işlem uzun sürüyor

In [33]:
routes = pd.read_csv('pause.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [38]:
routes[['route', 'distances']].head(10) # head ile sadece ilk 10 tanesini getiriyoruz.

,route,distances
0,12TH -> 16TH,13.736
1,12TH -> 19TH,568.000
2,12TH -> 24TH,14.114
3,12TH -> ANTC,48.119
4,12TH -> ASHB,5.460
5,12TH -> BALB,17.993
6,12TH -> BAYF,17.442
7,12TH -> BERY,59.775
8,12TH -> CAST,21.342
9,12TH -> CIVC,12.833


**Böylelikle her bir istasyonun birbiri arasındaki lineer mesafesini bulmuş olduk**